# QCA Comp Notebook 2

## Meeting Notes:
- Bloch Sphere
- Grover's

## Bloch Sphere (Introduction)

[QuTIP](http://qutip.org/docs/4.1/guide/guide-bloch.html) sports a python visualization library for displaying bloch sphere diagrams and representations.  It's incredibly extensible and supports animated simulations.  These quick exercises introduce you to the tool.

As review:
The Bloch sphere is a geometrical representation of the pure state space of a two-level quantum mechanical system (qubit).

The Bloch sphere is a unit 2-sphere, with antipodal points corresponding to a pair of mutually orthogonal state vectors. The north and south poles of the Bloch sphere are typically chosen to correspond to the standard basis vectors |0⟩ and |1⟩, respectively, which in turn might correspond e.g. to the spin-up and spin-down states of an electron. This choice is arbitrary, however. The points on the surface of the sphere correspond to the pure states of the system, whereas the interior points correspond to the mixed states.[2][3] The Bloch sphere may be generalized to an n-level quantum system.

Please refer to the documentation: http://qutip.org/docs/4.1/guide/guide-bloch.html



In [0]:
!pip3 install qutip
import qutip

In [0]:
from qutip import *
import matplotlib.pyplot as plt

In [0]:
q = basis(2,0)
b = Bloch()
## Exercise: plot vectors along the x,y,& z of the Bloch sphere (hint... use add_vectors)

b.show()

## Optional (but recommended) -- complete comp meeting exercises 

# Grover's Algorithm

Grover's is a famous quantum algorithm for "unstructured database search" ... a method of sublinear search in an unstructured data form.  Think of trying to find a integer x in an array [1, 3, 4, 75, 430003, 23, 12, ... ].  Can you design a classical algorithm to do this w/o touching every element? 

Grover's algorithm accomplishes this feat by first creating a uniform superposition over all possibilities and then destructively interfering states that would not provide a satisfying solution. 

Highly recommend this blog to review the concepts and intuition behind the algorithm:
http://twistedoakstudios.com/blog/Post2644_grovers-quantum-search-algorithm

Grover's algorithm is a quantum algorithm that finds with high probability the unique input to a black box function that produces a particular output value, using just $O({\sqrt {N}})$  evaluations of the function, where $N$ is the size of the function's domain. It was devised by Lov Grover in 1996. 

The analogous problem in classical computation cannot be solved in fewer than $ O(N)$ evaluations (because, in the worst case, the $N$-th member of the domain might be the correct member). At roughly the same time that Grover published his algorithm, Bennett, Bernstein, Brassard, and Vazirani proved that any quantum solution to the problem needs to evaluate the function  $\Omega ({\sqrt {N}})$ times, so Grover's algorithm is asymptotically optimal.

For reference, here's some more on the [theory.](https://quantumexperience.ng.bluemix.net/proxy/tutorial/full-user-guide/004-Quantum_Algorithms/070-Grover%27s_Algorithm.html)




---


Write a quantum program for the single-shot Grover’s algorithm in Pyquil.



The Python function you should produce is:




In [0]:
```
# data is an array of 0's and 1's such that there are exactly three times as many
# 0's as 1's.  e.g. data: [0, 0, 1,0]; Solution = 2;
def single_shot_grovers(data):
    # return an index that contains the value 1
```

In [0]:
import math
import numpy as np
import pyquil.quil as pq
from pyquil.gates import *
from pyquil.api import QVMConnection
from functools import reduce


def single_shot_grovers(data):
  
  # start the quantum program
	qvm = QVMConnection()
	p = pq.Program()

  # Grover's requires an N-dimensional state space H, where N is the number
  # of entries in the database. We can construct this with log_2(N) qubits.

  # Let data_size represent the number of qubits required. Use base change rule!
  # https://www.purplemath.com/modules/logrules5.htm
	data_size = int(math.log10(len(data)) / math.log10(2))
  
  # iterate over the qubits and build the state space.
  # allocate log_2(N) single-qubit Hadamard gates
	for index in range(0,data_size):
		p.inst(H(index))

  # Here we define the oracle: a gate that represents the predicate you want to 
  # search for a satisfying solution to. This gate should return true for 
  # the correct solution and false for all others. 
	oracle = np.identity(len(data)) - 2 * np.diag(data)
  
  # Make the oracle a static gate and add it for every qubit.
	p.defgate("oracle", oracle)
	p.inst(tuple(["oracle"] + [i for i in range(0,data_size)]))

  

    ## complete the following stub code (this is where you complete the code)
    ## This is the Grover diffusion operator.
	grover_op = 2 * (1./len(data)) * np.ones(tuple([len(data)] + [1])) * np.ones(len(data)) - np.identity(len(data))
  
  # TODO: define the grover gate, allocate it for every qubit
	p.defgate("oracle", oracle)
	p.inst(tuple(["oracle"] + [i for i in range(0,data_size)]))
  
  ## End of TODO


	for index in range(0,data_size):
		p.measure(index, index)

	print(p)
	wavefunction = qvm.wavefunction(p)
	print(qvm.wavefunction(p))
  
  # convert to classical index
	classical_regs = [i for i in range(0, data_size)]
	for i in classical_regs:
		p.measure(i,i)
	result = qvm.run(p, classical_regs)[0]
	result = reduce(lambda x,y: 2*x+y, result)
	return(result)

single_shot_grovers([0,0,1,0])